# Lec 9.1  DWave and qbsolv

This is a real industrial math problem. DWave is a company in Burnaby, BC that makes a quantum computing device, and they have released some demo software to show what their device can do.

We want to evaluate this software, and see what we can make it do.

## Issues.
 
1. Our Jupyter server at ucalgary.syzygy.ca is running slow, or not at all, at various times this week. This could be a network problem (even local Wifi might be an issue), it could be a load problem on the server, it could be a lack of disk space (as I have been hitting it pretty hard with a lot of files.) 

2. the software, qbsolv, is written in C and needs to be compiled. 

3. --- anything else? ----

I am tempted run this locally on my laptop, where I have no server issues, and I have a good C compiler. But this brings up another couple of issues:

4. My laptop only has Python 2 on it. And the IPython package is probably a year old. This might give me trouble.

5. My files on the laptop are pretty disorganized. I will move things around to consolidate.

## Getting the software

The software from DWave is available at this repository. 

https://github.com/dwavesystems/qbsolv

We can use GIT to clone it onto our system. I will do this in the following cell, but note that you could also do this from the terminal.


In [1]:
%%bash
git clone https://github.com/dwavesystems/qbsolv.git

Cloning into 'qbsolv'...


In [1]:
%%bash
ls

Lec9.1_qbsolv.ipynb
qbsolv


## Looking at the files.

All the qbsolv stuff comes in a folder called qbsolv. Some docs are pdf's, some are html, so you can't really read them from the Jupyter Notebook. 

Well, actually there is a way, using IFrame. Here it is:

In [5]:
# Docs on the code itself
from IPython.display import IFrame
IFrame("./qbsolv/doc/qbsolv.pdf", width=100, height=10)

In [6]:
# A technical report
from IPython.display import IFrame
IFrame("./qbsolv/qbsolv_techReport.pdf", width=100, height=10)

## Compiling the code.

Inside the source (src) directory there is a "make" file, which should compile the code for you.

To compile, just go into the terminal, cd into "src" and then enter the command "make"

Here is the result I got:
```
bash-3.2$ make
gcc -Ofast -Wall -Wextra -std=gnu99 -c -o debugs.o debugs.c
gcc -Ofast -Wall -Wextra -std=gnu99 -c -o dwsolv.o dwsolv.c
gcc -Ofast -Wall -Wextra -std=gnu99 -c -o main.o main.c
gcc -Ofast -Wall -Wextra -std=gnu99 -c -o readqubo.o readqubo.c
gcc -Ofast -Wall -Wextra -std=gnu99 -c -o solver.o solver.c
gcc -Ofast -Wall -Wextra -std=gnu99 -c -o util.o util.c
gcc -Ofast -Wall -Wextra -std=gnu99 -o qbsolv debugs.o dwsolv.o main.o readqubo.o solver.o util.o
```

You can see it compiled 6 smaller files, then combined them into one piece of code called qbsolv


## Running the code

Probably easiest to do this from the terminal.

cd into qbsolv/examples/mapColoringUSStates

type the command ./demoStates.sh      (which is a bash shell script)

It will give you a result like this:

``` 
bash-3.2$ ./demoStates.sh
204 Number of bits in solution
1000000000....
-51.00000 Energy of solution
0 Number of Partitioned calls
0.01081 seconds of classic cpu time
Solved -- Plotting
usa.qbout blank_US_Stat_map.svg
number of states 51
W  G  B  R  Y
0 14 12 10 15
```

Here is a picture of the output, the coloured USA map.

Note I am including a SVG file using a markdown cell in Jupyter.

![Alt text](qbsolv/examples/mapColoringUSStates/usa.qbout.xaa.svg "Optional title")

## How does this work?

The 51 states in the US are represented as 51 vertices on a graph.

Two vertices on the graph will share an edge iff the two corresponding states share a common border. (Corners don't count as borders.)

A vertex is given a color, represented by a 4 bit number. 0000, 0001, etc. But in principle, only one bit can take the value 1, the other three have value 0. 

The challenge is to find a set of colors for the graph so that no two adjacent vertices have the same color.

The file usa.adj is a text file defining the adjacency between US states.

The file adj2qubo.py is a python program converting the usa.adj adjacency into a QUBO file.

usa.qubo  is the QUBO file so generated, used as input to the qbsolv program.

color_state.py is a program that does the detailed coloring of the map, so we can look at it.


## Observations

I did a quick read-through of the source code, and I don't see anything "quantum-y" in there. The lack of complex numbers seems to be a clue that they aren't running a simulation of quantum annealing. It looks like they are simply doing a search by flipping bits between 0 and 1, looking locally for improvements (a binary tabu search). To me, it looks like a classic form of optimization, not really a simulation of quantum optimization.


I'll read some more, maybe I am missing something.

## Optimization problem

The DWave computer aims to find a solution to the quadratic unconstrained binary optimization problem (QUBO). Given an $n\times n$ upper triangular matrix $Q$ representing weights $Q_{ij}$ and binary vector $x \in \{0,1\}^n$ we define a function $f(x)$ as
$$f(x) = x^*Qx = \sum_i Q_{ii}x_i + \sum{i<j} Q_{ij}x_ix_j .$$

The QUBO problem is to find the minimizer for this function. That is, we wish to find
$$\min_x x^*Qx.$$

This problem is equivalent to several other optimization problems via simple transformations: the unconstrained binary quadratic program, the Ising model, and undirected graphical models.



## Examples

Why is this interesting? A simple example is a model for magnitization. Image a collection of magnetic particles, whose magnetic axis can only point up or down (1 or 0). Two particles that are close together will have more or less energy stored depending on whether their magnetic axis are parallel (both up, or both down), or antiparallel.

This energy can be represented by a quadratic term like
$$ Q_{12}(2x_1 -1) (2x_2 -1),$$
where $Q_{12}$ is a measure of the energy for the coupling of the two particles.

Notice that $(2x_1 -1)$ only takes values plus or minus one (since $x_1$ is only equal to zero or one), and so the product of the two factors is one when the particles are parallel, and minus one when they are anti-parallel. 

Do this for all particles $x_i,x_j$, using different weights $Q_{ij}$ which have something to do with the strength of interaction between two particles (e.g. maybe some are close together, some are far apart) and you get a sum
$$f(x) = \sum_{i<j} Q_{ij}(2x_i -1) (2x_j -1).$$

Some trivial transformations and you get it into QUBO form. 

A physicist would like to know the miminum energy state for this configuration of particles, hence we have a minimization problem.

## Graph coloring

A similar idea works for graph coloring. You have a graph with vertices and some edges. Maybeyou have 4 colors to choose from, so each vertex is represented by 4 bits, $x_1,x_2,x_3,x_4$. Only one bit can be equal to one, so assign a penalty if more than one bit at a vectex is non-zero. (Also assign a penalty if that are all zero.) If two vertices share an edge, assign a penalty if they have the same color. 

Thinking about these bits and penalties, you should be able to come up with a matrix $Q$ that represents all these penalties. 

Then minimize. 

## Coloring the USA
There are 50 states, and you need 4 colors. This suggest you need $4*50 = 200$ bits, but the program we ran used 204 bits. I wonder why? (District of Columbia perhaps is counted as the 51st state?)

The minimum solution had a minimum value of -51, which also seems to reflect something about how the penalty functions worked. (51 non-zero bits are need to select all the colors.)

## Using the code

The key to running the code is creating the qubo file, while contains all the information about the size of matrix Q and the values of the entries in Q. 

The trick for an optimization problem, then, is to take whatever problem you have and re-formulate it into the form of a quadratic binary optimization problem.

I can see this would be useful for graph-type problems, and other combinatorial problems. I don't know how I would use it for my typical continuous space, gradient descent type of optimization problem.

## Final comments

Reading DWave's technical documents that come with the software, they do indeed say this is a classical optimizer. There is no simulation of quantum behaviour in this software code.

They do say, if you want to see the quantum simulator in code, write to the authors. (I will do so.)

They also say, if you want to run your optimization on their DWave machine, write to the authors. If I get an interesting problem, I will do so!

Finally, note there are many limitations to the DWave machine. Their latest machine only has 2048 qubits. This means the matrix $Q$ can only be 2048 by 2048. Also, not all qubits can interact with any other qubit -- there is a limitation to which ones pairwise interact. This means only some of the off-diagonal entries in $Q$ can be non-zero. Moreover, there is limited precision on how accurately the value for each $Q_{ij}$ can be set -- it depends a lot on the physics of the real system they have constructed. 

Finally, the DWave system does not quarantee the right solution -- it only gives a correct solution to some level of probability. You can run the code many times to get many answers, and see if it is coming up with a good minimum often. 